In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy
import re
from collections import Counter

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import get_gene_and_cohort_list_utils
import configuration_util
filePathDict = configuration_util.get_all_files_path_dict()

In [30]:
"""def summarize_dominant_signatures_of_cases(mutClassificationDir = '/Users/friedman/Desktop/hypermutationStatusIds/'):
    d = {}
    for f in os.listdir(mutClassificationDir):
        cancerType = re.sub('.tsv', '', f)
        filePath = os.path.join(mutClassificationDir, f)
        df = pd.read_table(filePath)
        hypermutatedDf = df[df['hypermutantClassification'] == 'Hypermutated']
        domSigDict = dict(zip(hypermutatedDf['Tumor_Sample_Barcode'], hypermutatedDf['dominantSignature']))
        for tsb, domSig in domSigDict.items():
            d[tsb] = domSig
    return d
    """

#makes a dataframe of counts of oncogenic mutations
def make_counts_df(maf, dominantSignatureDict):
    
    def summarize_counts_for_mutation_type(oMaf, mutationType):
        
        oncogenes = get_gene_and_cohort_list_utils.get_oncogenes()
        tsgs = get_gene_and_cohort_list_utils.get_tsgs()
        truncatingConsequences = set(['Frame_Shift_Del', 'Frame_Shift_Ins', 'Nonsense_Mutation'])
        
        sMaf = None
        counts = None
        if mutationType == 'tsgTrunc':
            sMaf = oncogenicMaf[oncogenicMaf['Hugo_Symbol'].isin(tsgs) &
                (oncogenicMaf['Variant_Classification'].isin(truncatingConsequences))]
        if mutationType == 'oncogene':
            sMaf = oncogenicMaf[oncogenicMaf['Hugo_Symbol'].isin(oncogenes)]
        if mutationType == 'tsgMissense':
            sMaf = oncogenicMaf[oncogenicMaf['Hugo_Symbol'].isin(tsgs) &
                                (~oncogenicMaf['Variant_Classification'].isin(truncatingConsequences))]
            
        counts = Counter(sMaf['Tumor_Sample_Barcode'])
        counts = [(case, count) for case, count in counts.items()] + [(case, 0) for
            case in set(allIds) - set(sMaf['Tumor_Sample_Barcode'])] 
        return dict(counts)

    
    cancerTypeDict = dict(zip(maf['Tumor_Sample_Barcode'], maf['cancerType']))
    allIds = set(maf['Tumor_Sample_Barcode'])
    oncogenicMaf = maf[maf['oncogenic'].notnull()]
    
    tsgTruncatingCounts = summarize_counts_for_mutation_type(oncogenicMaf, 'tsgTrunc')
    oncogenicCounts = summarize_counts_for_mutation_type(oncogenicMaf, 'oncogene')
    tsgMissenseCounts = summarize_counts_for_mutation_type(oncogenicMaf, 'tsgMissense')
    listOfDicts = []
    for case in allIds:
        listOfDicts.append({'Tumor_Sample_Barcode': case, 'TsgTruncating': tsgTruncatingCounts[case],
                            'Oncogene': oncogenicCounts[case], 'TsgMissense': tsgMissenseCounts[case],
                            'cancerType': cancerTypeDict[case], 
                            'dominantSignature': dominantSignatureDict[case] if case in dominantSignatureDict else None
                           })
    return pd.DataFrame(listOfDicts)

def sample_normal_maf(normalCounts, hyperCounts, N=25):
    
    hyperCancerTypeCounter = Counter(hyperCounts['cancerType'])
    normalCancerTypeCounter = Counter(normalCounts['cancerType'])
    normalCounts['hypermutatedCount'] = normalCounts['cancerType'].apply(lambda x:
            1.0*hyperCancerTypeCounter[x]/normalCancerTypeCounter[x]
        if x in hyperCancerTypeCounter else 0)
    sampledCounts = normalCounts.sample(frac=N, weights='hypermutatedCount', replace=True)
    return sampledCounts
    

In [2]:
filePathDict = configuration_util.get_all_files_path_dict()
writeDir = '/Users/friedman/Desktop/hypermutationProjectFinal/scripts/figure1/FIGURE1_PLOTTING_FILES'

In [12]:
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])
dominantSignatureDict = get_gene_and_cohort_list_utils.get_pan_impact_signature_mapping(
    filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'])
allHypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
allNormalIds = get_gene_and_cohort_list_utils.get_all_normal_ids(hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
cancerTypeDict = get_gene_and_cohort_list_utils.get_impact_cancer_type_info(impactCancerTypeInfoPath = filePathDict['CANCER_TYPE_INFO'])


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:163: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  impactCancerTypeDf = pd.read_table(impactCancerTypeInfoPath)


In [13]:
allImpactMutsMaf['cancerType'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)
hypermutatedMaf = allImpactMutsMaf[allImpactMutsMaf['Tumor_Sample_Barcode'].isin(allHypermutantIds)]
nonHypermutatedMaf = allImpactMutsMaf[allImpactMutsMaf['Tumor_Sample_Barcode'].isin(allNormalIds)]

In [31]:
dfCountsNormal = make_counts_df(nonHypermutatedMaf, dominantSignatureDict)
dfCountsHypermutated = make_counts_df(hypermutatedMaf, dominantSignatureDict)

In [15]:
dfCountsNormalSampled = sample_normal_maf(dfCountsNormal, dfCountsHypermutated)

In [16]:
dfCountsHypermutated['burdenType'] = 'hypermutated'
dfCountsNormalSampled['burdenType'] = 'normal'
dfCombined = pd.concat([dfCountsHypermutated, dfCountsNormalSampled])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
meltedDf = pd.melt(dfCombined, id_vars=['Tumor_Sample_Barcode', 'burdenType'], value_vars=['Oncogene', 'TsgMissense', 'TsgTruncating'])


In [18]:
#ggplot sturggles for some reason so I just create a summary df
listOfDicts = []
for variable in set(meltedDf['variable']):
    hypermutatedData = meltedDf[meltedDf['burdenType'] == 'hypermutated']
    nonHypermutatedData = meltedDf[meltedDf['burdenType'] != 'hypermutated']
    listOfDicts.append({
        'frac': sum(hypermutatedData[hypermutatedData['variable'] == variable]['value'])/1.0*sum(hypermutatedData['value']),
        'burdenType': 'hypermutated', 'mutType': variable
    })
    listOfDicts.append({
        'frac': sum(nonHypermutatedData[nonHypermutatedData['variable'] == variable]['value'])/1.0*sum(nonHypermutatedData['value']),
        'burdenType': 'non-hypermutated', 'mutType': variable
    })
summaryDf = pd.DataFrame(listOfDicts)

In [28]:
summaryDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/figure2aGeneTypes.tsv', index=False, sep='\t')

In [118]:
#
##
####
#########
def attribute_mutations_to_signatures(maf, mutAttributionDf):
    dictOfDicts = {}
    for case in set(mutAttributionDf['Sample Name']):
        dictOfDicts[case] = dict(mutAttributionDf[mutAttributionDf['Sample Name'] == case].iloc[0])
    possibleQuadNucs = [firstLetter + change + lastLetter for firstLetter in ['A', 'C', 'T', 'G']
    for change in ['CA', 'CG', 'CT', 'TA', 'TG', 'TC'] for lastLetter in ['A', 'C', 'T', 'G']]

    print 'attributing mutations'
    maf['mutAttribution'] = maf.apply(lambda row: dictOfDicts[row['Tumor_Sample_Barcode']][row['quadNuc']]
     if row['quadNuc'] in possibleQuadNucs and row['Tumor_Sample_Barcode'] in dictOfDicts else None, axis=1)
    return maf


#SUMMARIZE INFO ABOUT MUTATION ATTRIBUTION

In [134]:
oncogenes = get_gene_and_cohort_list_utils.get_oncogenes()
tsgs = get_gene_and_cohort_list_utils.get_tsgs()
truncatingConsequences = set(['Frame_Shift_Del', 'Frame_Shift_Ins', 'Nonsense_Mutation'])  
    
oncogenicMutAttributionMaf = mutAttributionMaf[mutAttributionMaf['oncogenic'].notnull()]
oncogenicMutAttributionMaf['geneType'] = oncogenicMutAttributionMaf.apply(lambda row:
        'oncogene' if row['Hugo_Symbol'] in oncogenes
        else 'tsgTruncating' if row['Variant_Classification'] in truncatingConsequences
        else 'tsgMissense', axis=1)

finalDf = oncogenicMutAttributionMaf[['geneType', 'hypermutationInduced']]
finalDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/mutAttributionData.tsv', index=False, sep='\t')

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Supplemental Figure: signature specific mutation type breakdown for hypermutated tumors

In [5]:
hypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(filePathDict['HYPERMUTATION_STATUS_IDS'])
signaturesInfo = get_gene_and_cohort_list_utils.get_hypermutator_signature_cohorts(
    impactSigsPath=filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'])


/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:170: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  df = pd.read_table(path)
/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:150: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  impactSigs = pd.read_table(impactSigsPath)


In [45]:
meltedDf2 = pd.melt(dfCountsHypermutated, id_vars=['dominantSignature'], value_vars=['Oncogene', 'TsgMissense', 'TsgTruncating'])

In [59]:
#todo write this more efficiently
meltedDf2['dominantSignatureAdj'] = meltedDf2['dominantSignature'].apply(lambda x:
    '_MMR' if x == 'mean_1' or x == 'mean_MMR'
    else '_SMOKING' if x == 'mean_SMOKING'
    else '_APOBEC' if x == 'APOBEC'
    else '_POLE' if x == 'mean_10'
    else '_TMZ' if x == 'mean_11'
    else '_UV' if x == 'mean_7'
    else 'other')

meltedDf2['frac'] = meltedDf2.apply(lambda row:
    1.0*row['value']/sum(meltedDf2[meltedDf2['dominantSignatureAdj'] == row['dominantSignatureAdj']]['value']), axis=1)


In [60]:
meltedDf2.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/fig2aSupplement.tsv', index=False, sep='\t')